In [2]:
import pandas as pd
from pathlib import Path

# 📁 Définition des chemins
PROJECT_ROOT = Path("..").resolve()
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"
DATA_OUTPUTS = PROJECT_ROOT / "data" / "outputs"

# 📄 Chargement du fichier nettoyé
df = pd.read_csv(DATA_PROCESSED / "processed_data.csv")

# --- 1. Besoins textuels (long format) ---
besoin_text_cols = [
    "amelioration_pratiques",
    "besoins_aide_evaluation",
    "besoins_diversite_classe",
    "besoins_competence_numerique",
    "renforcement_collaboration",
    "preference_accompagnement",
    "besoin_dev_professionnel"
]

df_text = df[["enseignant_uid"] + besoin_text_cols].melt(
    id_vars="enseignant_uid",
    var_name="question_source",
    value_name="besoin_brut"
)
df_text["besoin_brut"] = df_text["besoin_brut"].astype(str).str.strip()
df_text = df_text[df_text["besoin_brut"] != ""].dropna()
df_text["categorie_besoin"] = None  # à compléter par classification

# --- 2. Besoins MLF (besoin fort/faible) ---
besoin_mlf_cols = [
    "besoin_formation_plurilingue",
    "besoin_formation_transversales",
    "besoin_formation_maternelle",
    "besoin_formation_orientation",
    "besoin_formation_stiam"
]

df_mlf = df[["enseignant_uid"] + besoin_mlf_cols].copy()
df_mlf = df_mlf.melt(id_vars="enseignant_uid", var_name="question_source", value_name="besoin_brut")

# On garde uniquement les réponses « Besoin fort »
df_mlf = df_mlf[df_mlf["besoin_brut"] == "besoin fort"].copy()

# On catégorise directement selon le thème MLF (extrait depuis le nom de colonne)
df_mlf["categorie_besoin"] = df_mlf["question_source"].str.replace("besoin_formation_", "", regex=False).str.lower()

# --- 3. Fusion finale ---
df_besoins = pd.concat([df_text, df_mlf], ignore_index=True)

# 🔁 Réorganisation des colonnes
df_besoins = df_besoins[["enseignant_uid", "question_source", "besoin_brut", "categorie_besoin"]]

# Export
df_besoins.to_csv(DATA_OUTPUTS / "df_besoins.csv", index=False)

# Aperçu
print(f"✅ df_besoins global exporté avec {df_besoins.shape[0]} lignes")
df_besoins.head(60)


✅ df_besoins global exporté avec 23 lignes


,enseignant_uid,question_source,besoin_brut,categorie_besoin
0,E0001,amelioration_pratiques,Les outils informatiques et enseigner sur 3 ni...,None
1,E0002,amelioration_pratiques,"Ma capacité à enseigner dans une autre langue,...",None
2,E0003,amelioration_pratiques,Enrichir le contenu dans certaines disciplines...,None
3,E0001,besoins_aide_evaluation,Peut-être les outils informatiques Je ne sais...,None
4,E0002,besoins_aide_evaluation,Je me débrouille merci j'ai des collègues sur...,None
5,E0003,besoins_aide_evaluation,"En EPS, le travail d'équipe est efficace, les ...",None
6,E0001,besoins_diversite_classe,De formations en présentiel et d'un suivi de c...,None
7,E0002,besoins_diversite_classe,"Je parle darija, je comprendsconnais le contex...",None
8,E0003,besoins_diversite_classe,Avoir plus de soutien sur la transmission des ...,None
9,E0001,besoins_competence_numerique,Oui Le TBI et un peu tout en général Connaître...,None


In [ ]:
# import time
# from tqdm import tqdm
# from dotenv import load_dotenv
# import os
# from openai import OpenAI


# # Charger les variables depuis le fichier .env
# load_dotenv()

# # Clé API OpenAI
# api_key = os.getenv('OPENAI_API_KEY')
# client = OpenAI(api_key = api_key)

# def classer_besoin_gpt(besoin, question, client):
#     prompt = (
#         "Voici une réponse exprimée par un enseignant à la question suivante :\n"
#         f"« {question} »\n\n"
#         f"Réponse : « {besoin} »\n\n"
#         "Classe cette réponse dans une seule catégorie thématique courte (1 à 4 mots).\n"
#         "Ne réponds que par le nom de la catégorie, sans phrase ni ponctuation.\n"
#     )

#     try:
#         response = client.chat.completions.create(
#             model="gpt-4o",  # ou "gpt-4" si tu veux
#             messages=[{"role": "user", "content": prompt}],
#             temperature=0.2,
#         )
#         return response.choices[0].message.content.strip()
#     except Exception as e:
#         print(f"Erreur : {e}")
#         return None

# # On ne classe que les lignes encore non catégorisées
# df_besoins_to_classify = df_besoins[df_besoins["categorie_besoin"].isna()].copy()

# # Application ligne par ligne
# categories = []
# for _, row in tqdm(df_besoins_to_classify.iterrows(), total=len(df_besoins_to_classify)):
#     categorie = classer_besoin_gpt(row["besoin_brut"], row["question_source"], client)
#     categories.append(categorie)
#     time.sleep(1.2)  # pour respecter les quotas si tu n’as pas GPT-4o avec débit élevé

# # Insertion des résultats dans le DataFrame original
# df_besoins.loc[df_besoins["categorie_besoin"].isna(), "categorie_besoin"] = categories

# # Export du fichier enrichi
# df_besoins.to_csv(DATA_OUTPUTS / "df_besoins.csv", index=False)

# print("✅ Catégorisation terminée et exportée.")
# df_besoins.sample(5)


100%|██████████| 21/21 [00:39<00:00,  1.88s/it]

✅ Catégorisation terminée et exportée.


,enseignant_uid,question_source,besoin_brut,categorie_besoin
6,E0001,besoins_diversite_classe,De formations en présentiel et d'un suivi de c...,Formation continue
5,E0003,besoins_aide_evaluation,"En EPS, le travail d'équipe est efficace, les ...",Travail d'équipe
2,E0003,amelioration_pratiques,Enrichir le contenu dans certaines disciplines...,Amélioration pédagogique
18,E0001,besoin_dev_professionnel,Les outils informatiques et l'enseignement sur...,Formation numérique
7,E0002,besoins_diversite_classe,"Je parle darija, je comprendsconnais le contex...",Compétences linguistiques et culturelles


In [ ]:
from sentence_transformers import SentenceTransformer
# model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')


In [15]:
model.save("../models/paraphrase_multilingual")


In [16]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]
embeddings = model.encode(sentences)
print(embeddings)


[[ 0.18845697  0.1742564   0.05447777  0.29051754  0.16766407 -0.04720681
   0.64558     0.15980875  0.22689267 -0.03089049  0.25588372 -0.05258765
  -0.2261013  -0.05710638  0.13042611  0.12495332  0.31749615  0.1944441
  -0.58632565 -0.01258591  0.6099091   0.16432753  0.03331159 -0.27383074
  -0.28975758 -0.21119703 -0.02261396 -0.1703594   0.16159002  0.06082742
  -0.24162391  0.18579209  0.42740947  0.19295181 -0.07234471  0.16611099
   0.10442821  0.20477232  0.21116705  0.19974013 -0.09408273 -0.17383671
   0.06427342  0.28025505 -0.29530567  0.06209521  0.10427673 -0.02364426
   0.12913169 -0.12617457 -0.17899014  0.03700579 -0.6125062   0.05029835
   0.17730355  0.22494124  0.1738607  -0.03840288 -0.21286817  0.2584925
  -0.12101631  0.30971518 -0.4196635   0.00907677  0.14188926 -0.30556944
   0.17621139 -0.07087342 -0.6203312   0.6770835   0.01723732  0.18405105
  -0.16785756  0.20452651 -0.14770278 -0.06175341  0.6301743   0.11120182
   0.05153079  0.15927401 -0.05370891  0

In [ ]:
embeddings = model.encode(besoins_textuels)

# 📉 3. Réduction de dimension pour visualisation
umap_model = umap.UMAP(n_neighbors=15, n_components=2, min_dist=0.0, metric='cosine')
umap_embeddings = umap_model.fit_transform(embeddings)

# 🔍 4. Clustering non supervisé
clusterer = hdbscan.HDBSCAN(min_cluster_size=5, metric='euclidean', prediction_data=True)
cluster_labels = clusterer.fit_predict(umap_embeddings)

# 🧩 5. Attribution des clusters dans le dataframe
df_clusters = pd.DataFrame({
    "besoin_brut": besoins_textuels,
    "cluster": cluster_labels,
    "x": umap_embeddings[:, 0],
    "y": umap_embeddings[:, 1]
})

# 📊 6. Visualisation 2D
plt.figure(figsize=(10, 8))
sns.scatterplot(data=df_clusters, x="x", y="y", hue="cluster", palette="tab10", legend="full")
plt.title("Clustering des besoins exprimés (UMAP + HDBSCAN)")
plt.xlabel("UMAP-1")
plt.ylabel("UMAP-2")
plt.legend(title="Cluster")
plt.show()